<a href="https://colab.research.google.com/github/samirgadkari/DS-Unit-4-Sprint-3-Neural-Networks/blob/master/DS43SC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Networks Sprint Challenge

## 1) Define the following terms:

- Neuron
- Input Layer
- Hidden Layer
- Output Layer
- Activation
- Backpropagation

  - Neuron:
In machine learning, a neuron is a function that takes multiple inputs and outputs one value. If the neuron is in the first hidden layer, it's inputs are all the feature values of the input layer + the bias value. If the neuron is in a deeper hidden layer, it's inputs are all the neurons to it's left + the bias value. Each neuron multiplies the weights and inputs coming into it, adds the bias, and applies the activation function to the result to get the output value of the neuron.


  - Input Layer:
Input layer contains a set of cells. Each cell represents one feature, and contains the value present in that feature for that observation.


  - Hidden Layer:
Hidden layers are all the layers to the right of the input layer. These are called hidden, because we cannot directly influence them.


  - Output Layer:
An output layer contains as many neurons as you expect to solve your particular machine-learning problem. The output layer is activated by an activation function specific to the problem. For example for regression, the activation function is the linear function. The output values will be unbounded with this function, which is what you want. For binary classfication, you might use the sigmoid function to restrict values between 0 and 1. These values are then the probability of predicting the primary class.


  - Activation Function (also called Transfer Function):
An activation function takes a value and returns another value. It has to be differentiable, since we will differentiate it during the backpropagation step. You may choose different activation functions for different layers. An activation function is applied after a neuron has weighed all input values, added them together, and then added a bias.
Common activation functions are the Sigmoid, tanh, step, and relu.

  - Backpropagation
On each iteration, we will get an output value. Backpropagation is the step that updates the weights/bias in the correct direction so that our error between the output value and the training output value is lower than the last error.

## 2) Create a perceptron class that can model the behavior of an AND gate. You can use the following table as your training data:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 1  | 1  | 1  | 1 |
| 1  | 0  | 1  | 0 |
| 0  | 1  | 1  | 0 |
| 0  | 0  | 1  | 0 |

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import StratifiedKFold, \
                                    cross_val_score, \
                                    GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from keras.wrappers.scikit_learn import KerasClassifier

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

Using TensorFlow backend.


In [0]:
X = np.array([[1, 1, 1], 
              [1, 0, 1],
              [0, 1, 1], 
              [0, 0, 1]])
y = np.array([[1], 
              [0], 
              [0], 
              [0]])
df = pd.DataFrame(data=np.concatenate([X, y], axis=1))
df

,0,1,2,3
0,1,1,1,1
1,1,0,1,0
2,0,1,1,0
3,0,0,1,0


In [0]:
class Perceptron:
    def __init__(self, inputs, weights, bias, outputs):
        self.inputs = inputs
        self.weights = weights
        self.outputs = outputs
    
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    # Explanation for why this is is given here:
    # https://towardsdatascience.com/derivative-of-the-sigmoid-function-536880cf918e
    def sigmoid_derivative(x):
        return sigmoid(x) * (1 - sigmoid(x))
    
    def iterate(self, num_iters):
        for iteration in range(num_iters):

          # Weighted sum of inputs and weights
          weighted_sum = np.dot(self.inputs, 
                                self.weights) + bias

          # Activate with sigmoid function
          activated_output = sigmoid(weighted_sum)

          # Calculate Error
          error = self.outputs - activated_output

          # Calculate weight adjustments with sigmoid_derivative
          adjustments = error * sigmoid_derivative(activated_output)

          # Update weights
          self.weights += np.dot(self.inputs.T, adjustments)

        print('optimized weights after training: ')
        print(weights)

        np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
        print('activated_output:', activated_output)

        return activated_output

In [0]:
inputs  = X
weights = 2 * np.random.random((3, 1)) - 1
bias    = 1
correct_outputs = y
perceptron = Perceptron(inputs, weights, bias, correct_outputs)
perceptron.iterate(10000)

optimized weights after training: 
[[11.839]
 [11.839]
 [-19.048]]
activated_output: [[0.996]
 [0.002]
 [0.002]
 [0.000]]


array([[0.996],
       [0.002],
       [0.002],
       [0.000]])

## 3) Implement a Neural Network Multilayer Perceptron class that uses backpropagation to update the network's weights. 
- Your network must have one hidden layer. 
- You do not have to update weights via gradient descent. You can use something like the derivative of the sigmoid function to update weights.
- Train your model on the Heart Disease dataset from UCI:

[Github Dataset](https://github.com/ryanleeallred/datasets/blob/master/heart.csv)

[Raw File on Github](https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv)


In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv',
                 header=0)
print(df.shape)
print(df.dtypes)
df.head()

(303, 14)
age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
target        int64
dtype: object


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [0]:
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [0]:
for col in df.columns:
  if df[col].dtype == 'int64':
    df[col] = df[col].astype('float')

In [0]:
df.dtypes

age         float64
sex         float64
cp          float64
trestbps    float64
chol        float64
fbs         float64
restecg     float64
thalach     float64
exang       float64
oldpeak     float64
slope       float64
ca          float64
thal        float64
target      float64
dtype: object

In [0]:
df_X = df.drop(['target'], axis=1)
df_y = df['target']
df_X.shape, df_y.shape

((303, 13), (303,))

In [0]:
seed = 101
np.random.seed(seed)

class Neural_Network(object):
    def __init__(self, num_input_nodes, num_hidden_nodes, num_output_nodes, learning_rate):
        self.num_input_nodes  = num_input_nodes   # 13 input nodes
        self.num_hidden_nodes = num_hidden_nodes  # 8 hidden nodes
        self.num_output_nodes = num_output_nodes  # 1 output node
        self.learning_rate = learning_rate
        
        self.L1_weights = np.random.randn(num_input_nodes, num_hidden_nodes)
        self.L2_weights = np.random.randn(num_hidden_nodes, num_output_nodes)
        
    def feed_forward(self, X):
        self.hidden_sum = np.dot(X, self.L1_weights)
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        self.output_sum = np.dot(self.activated_hidden, self.L2_weights)
        self.activated_output = self.sigmoid(self.output_sum)
        return self.activated_output
        
    def sigmoid(self, s):
        return 1 / (1 + np.exp(-s))
    
    def sigmoid_prime(self, s):
        return s * (1 - s)
    
    def backward(self, X, y, o):

        self.o_error = y - o
        self.o_delta = self.o_error * self.sigmoid_prime(o)
        
        self.z2_error = self.o_delta.dot(self.L2_weights.T)
        self.z2_delta = self.z2_error * self.sigmoid_prime(self.activated_hidden)
        
        # Adjust weights
        self.L1_weights += self.learning_rate * X.T.dot(self.z2_delta)
        self.L2_weights += self.learning_rate * self.activated_hidden.T.dot(self.o_delta)

    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)

In [0]:
X = df_X.values
y = df_y.values
y_out = []
for i in range(len(y)):
    y_out.append([y[i]])
y = y_out.copy()

In [0]:
NN = Neural_Network(13, 8, 1, 0.01)
for i in range(100000): # trains the NN 1,000 times
  if i+1 in [1,2,3,4,5] or (i+1) % 50 == 0:
    print('+---------- EPOCH', i+1, '-----------+', end=' ')
#     print("Input: \n", X) 
#     print("Actual Output: \n", y)  
#     print("Predicted Output: \n" + str(NN.feed_forward(X))) 
    print("Loss: " + str(np.mean(np.square(y - NN.feed_forward(X))))) # mean sum squared loss
#     print("\n")
  NN.train(X, y)

+---------- EPOCH 1 -----------+ Loss: 0.49187528649037016
+---------- EPOCH 2 -----------+ Loss: 0.4762493648106615
+---------- EPOCH 3 -----------+ Loss: 0.4525266892820536
+---------- EPOCH 4 -----------+ Loss: 0.41377119210066793
+---------- EPOCH 5 -----------+ Loss: 0.3527827310980319
+---------- EPOCH 50 -----------+ Loss: 0.24741927143550208
+---------- EPOCH 100 -----------+ Loss: 0.24741797478714142
+---------- EPOCH 150 -----------+ Loss: 0.24741671096803017
+---------- EPOCH 200 -----------+ Loss: 0.24741547881916798
+---------- EPOCH 250 -----------+ Loss: 0.24741427723310577
+---------- EPOCH 300 -----------+ Loss: 0.24741310515121642
+---------- EPOCH 350 -----------+ Loss: 0.24741196156113096
+---------- EPOCH 400 -----------+ Loss: 0.24741084549432832
+---------- EPOCH 450 -----------+ Loss: 0.247409756023871
+---------- EPOCH 500 -----------+ Loss: 0.24740869226227383
+---------- EPOCH 550 -----------+ Loss: 0.24740765335949932
+---------- EPOCH 600 -----------+ Loss:

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: overflow encountered in exp


+---------- EPOCH 850 -----------+ Loss: 0.2474018986408573
+---------- EPOCH 900 -----------+ Loss: 0.24740101272411655
+---------- EPOCH 950 -----------+ Loss: 0.24740014600879012
+---------- EPOCH 1000 -----------+ Loss: 0.24739929790869075
+---------- EPOCH 1050 -----------+ Loss: 0.24739846786037356
+---------- EPOCH 1100 -----------+ Loss: 0.24739765532207764
+---------- EPOCH 1150 -----------+ Loss: 0.24739685977272374
+---------- EPOCH 1200 -----------+ Loss: 0.2473960807109653
+---------- EPOCH 1250 -----------+ Loss: 0.24739531765429104
+---------- EPOCH 1300 -----------+ Loss: 0.24739457013817348
+---------- EPOCH 1350 -----------+ Loss: 0.24739383771526288
+---------- EPOCH 1400 -----------+ Loss: 0.24739311995462357
+---------- EPOCH 1450 -----------+ Loss: 0.2473924164410081
+---------- EPOCH 1500 -----------+ Loss: 0.2473917267741699
+---------- EPOCH 1550 -----------+ Loss: 0.24739105056821079
+---------- EPOCH 1600 -----------+ Loss: 0.2473903874509609
+---------- EPOC

+---------- EPOCH 7900 -----------+ Loss: 0.2473538050503346
+---------- EPOCH 7950 -----------+ Loss: 0.2473536803308179
+---------- EPOCH 8000 -----------+ Loss: 0.24735355675540704
+---------- EPOCH 8050 -----------+ Loss: 0.24735343430884413
+---------- EPOCH 8100 -----------+ Loss: 0.24735331297613697
+---------- EPOCH 8150 -----------+ Loss: 0.24735319274255246
+---------- EPOCH 8200 -----------+ Loss: 0.2473530735936119
+---------- EPOCH 8250 -----------+ Loss: 0.24735295551508518
+---------- EPOCH 8300 -----------+ Loss: 0.24735283849298573
+---------- EPOCH 8350 -----------+ Loss: 0.2473527225135652
+---------- EPOCH 8400 -----------+ Loss: 0.2473526075633088
+---------- EPOCH 8450 -----------+ Loss: 0.24735249362893
+---------- EPOCH 8500 -----------+ Loss: 0.2473523806973663
+---------- EPOCH 8550 -----------+ Loss: 0.24735226875577399
+---------- EPOCH 8600 -----------+ Loss: 0.24735215779152417
+---------- EPOCH 8650 -----------+ Loss: 0.24735204779219816
+---------- EPOCH

+---------- EPOCH 14750 -----------+ Loss: 0.2473433494696931
+---------- EPOCH 14800 -----------+ Loss: 0.24734330338301577
+---------- EPOCH 14850 -----------+ Loss: 0.24734325756299178
+---------- EPOCH 14900 -----------+ Loss: 0.24734321200735865
+---------- EPOCH 14950 -----------+ Loss: 0.2473431667138792
+---------- EPOCH 15000 -----------+ Loss: 0.24734312168034078
+---------- EPOCH 15050 -----------+ Loss: 0.24734307690455554
+---------- EPOCH 15100 -----------+ Loss: 0.24734303238435965
+---------- EPOCH 15150 -----------+ Loss: 0.24734298811761302
+---------- EPOCH 15200 -----------+ Loss: 0.24734294410219945
+---------- EPOCH 15250 -----------+ Loss: 0.2473429003360255
+---------- EPOCH 15300 -----------+ Loss: 0.24734285681702092
+---------- EPOCH 15350 -----------+ Loss: 0.24734281354313778
+---------- EPOCH 15400 -----------+ Loss: 0.24734277051235073
+---------- EPOCH 15450 -----------+ Loss: 0.2473427277226562
+---------- EPOCH 15500 -----------+ Loss: 0.24734268517207

...

+---------- EPOCH 69250 -----------+ Loss: 0.24733147170495587
+---------- EPOCH 69300 -----------+ Loss: 0.24733146920636156
+---------- EPOCH 69350 -----------+ Loss: 0.2473314667113468
+---------- EPOCH 69400 -----------+ Loss: 0.24733146421990387
+---------- EPOCH 69450 -----------+ Loss: 0.24733146173202525
+---------- EPOCH 69500 -----------+ Loss: 0.24733145924770328
+---------- EPOCH 69550 -----------+ Loss: 0.2473314567669305
+---------- EPOCH 69600 -----------+ Loss: 0.24733145428969927
+---------- EPOCH 69650 -----------+ Loss: 0.2473314518160021
+---------- EPOCH 69700 -----------+ Loss: 0.2473314493458316
+---------- EPOCH 69750 -----------+ Loss: 0.2473314468791802
+---------- EPOCH 69800 -----------+ Loss: 0.24733144441604046
+---------- EPOCH 69850 -----------+ Loss: 0.247331441956405
+---------- EPOCH 69900 -----------+ Loss: 0.2473314395002664
+---------- EPOCH 69950 -----------+ Loss: 0.24733143704761718
+---------- EPOCH 70000 -----------+ Loss: 0.24733143459845014


+---------- EPOCH 76050 -----------+ Loss: 0.24733116189942797
+---------- EPOCH 76100 -----------+ Loss: 0.24733115982557832
+---------- EPOCH 76150 -----------+ Loss: 0.24733115775444212
+---------- EPOCH 76200 -----------+ Loss: 0.24733115568601394
+---------- EPOCH 76250 -----------+ Loss: 0.2473311536202885
+---------- EPOCH 76300 -----------+ Loss: 0.24733115155726068
+---------- EPOCH 76350 -----------+ Loss: 0.2473311494969252
+---------- EPOCH 76400 -----------+ Loss: 0.24733114743927673
+---------- EPOCH 76450 -----------+ Loss: 0.24733114538431022
+---------- EPOCH 76500 -----------+ Loss: 0.24733114333202033
+---------- EPOCH 76550 -----------+ Loss: 0.24733114128240197
+---------- EPOCH 76600 -----------+ Loss: 0.24733113923544983
+---------- EPOCH 76650 -----------+ Loss: 0.247331137191159
+---------- EPOCH 76700 -----------+ Loss: 0.2473311351495241
+---------- EPOCH 76750 -----------+ Loss: 0.2473311331105401
+---------- EPOCH 76800 -----------+ Loss: 0.2473311310742019

+---------- EPOCH 82850 -----------+ Loss: 0.2473309027711855
+---------- EPOCH 82900 -----------+ Loss: 0.24733090102293764
+---------- EPOCH 82950 -----------+ Loss: 0.2473308992767943
+---------- EPOCH 83000 -----------+ Loss: 0.24733089753275186
+---------- EPOCH 83050 -----------+ Loss: 0.24733089579080636
+---------- EPOCH 83100 -----------+ Loss: 0.2473308940509543
+---------- EPOCH 83150 -----------+ Loss: 0.24733089231319164
+---------- EPOCH 83200 -----------+ Loss: 0.24733089057751495
+---------- EPOCH 83250 -----------+ Loss: 0.2473308888439204
+---------- EPOCH 83300 -----------+ Loss: 0.24733088711240425
+---------- EPOCH 83350 -----------+ Loss: 0.24733088538296275
+---------- EPOCH 83400 -----------+ Loss: 0.24733088365559214
+---------- EPOCH 83450 -----------+ Loss: 0.24733088193028901
+---------- EPOCH 83500 -----------+ Loss: 0.24733088020704946
+---------- EPOCH 83550 -----------+ Loss: 0.24733087848586974
+---------- EPOCH 83600 -----------+ Loss: 0.24733087676674

+---------- EPOCH 89650 -----------+ Loss: 0.24733068289756766
+---------- EPOCH 89700 -----------+ Loss: 0.24733068140428915
+---------- EPOCH 89750 -----------+ Loss: 0.24733067991267507
+---------- EPOCH 89800 -----------+ Loss: 0.24733067842272277
+---------- EPOCH 89850 -----------+ Loss: 0.24733067693442948
+---------- EPOCH 89900 -----------+ Loss: 0.24733067544779247
+---------- EPOCH 89950 -----------+ Loss: 0.24733067396280894
+---------- EPOCH 90000 -----------+ Loss: 0.24733067247947632
+---------- EPOCH 90050 -----------+ Loss: 0.24733067099779163
+---------- EPOCH 90100 -----------+ Loss: 0.24733066951775237
+---------- EPOCH 90150 -----------+ Loss: 0.2473306680393556
+---------- EPOCH 90200 -----------+ Loss: 0.24733066656259875
+---------- EPOCH 90250 -----------+ Loss: 0.24733066508747917
+---------- EPOCH 90300 -----------+ Loss: 0.247330663613994
+---------- EPOCH 90350 -----------+ Loss: 0.24733066214214064
+---------- EPOCH 90400 -----------+ Loss: 0.2473306606719

+---------- EPOCH 96450 -----------+ Loss: 0.24733049403721308
+---------- EPOCH 96500 -----------+ Loss: 0.24733049274724778
+---------- EPOCH 96550 -----------+ Loss: 0.247330491458621
+---------- EPOCH 96600 -----------+ Loss: 0.2473304901713308
+---------- EPOCH 96650 -----------+ Loss: 0.24733048888537518
+---------- EPOCH 96700 -----------+ Loss: 0.247330487600752
+---------- EPOCH 96750 -----------+ Loss: 0.24733048631745932
+---------- EPOCH 96800 -----------+ Loss: 0.24733048503549493
+---------- EPOCH 96850 -----------+ Loss: 0.24733048375485694
+---------- EPOCH 96900 -----------+ Loss: 0.24733048247554318
+---------- EPOCH 96950 -----------+ Loss: 0.24733048119755166
+---------- EPOCH 97000 -----------+ Loss: 0.24733047992088036
+---------- EPOCH 97050 -----------+ Loss: 0.24733047864552718
+---------- EPOCH 97100 -----------+ Loss: 0.24733047737149022
+---------- EPOCH 97150 -----------+ Loss: 0.24733047609876735
+---------- EPOCH 97200 -----------+ Loss: 0.247330474827356

## 4) Implement a Multilayer Perceptron architecture of your choosing using the Keras library. Train your model and report its baseline accuracy. Then hyperparameter tune at least two parameters and report your model's accuracy. 

- Use the Heart Disease Dataset (binary classification)
- Use an appropriate loss function for a binary classification task
- Use an appropriate activation function on the final layer of your network. 
- Train your model using verbose output for ease of grading.
- Use GridSearchCV to hyperparameter tune your model. (for at least two hyperparameters)
- When hyperparameter tuning, show you work by adding code cells for each new experiment. 
- Report the accuracy for each combination of hyperparameters as you test them so that we can easily see which resulted in the highest accuracy.
- You must hyperparameter tune at least 5 parameters in order to get a 3 on this section.

In [0]:
def baseline_model():
  model = Sequential()
  model.add(Dense(13, input_dim=X.shape[1], activation='relu'))
  model.add(Dense(8, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

seed = 101
np.random.seed(seed)
estimator = KerasClassifier(build_fn=baseline_model, 
                                     epochs=150,
                                     batch_size=32,
                                     verbose=1)
kfold = StratifiedKFold(n_splits=5, random_state=seed)
results = cross_val_score(estimator, X, y, cv=kfold)

Epoch 1/150
242/242 [==============================] - 1s 4ms/step - loss: 6.9056 - acc: 0.5455
Epoch 2/150
242/242 [==============================] - 0s 62us/step - loss: 6.1594 - acc: 0.5413
Epoch 3/150
242/242 [==============================] - 0s 60us/step - loss: 4.9671 - acc: 0.5744
Epoch 4/150
242/242 [==============================] - 0s 75us/step - loss: 3.9814 - acc: 0.5124
Epoch 5/150
242/242 [==============================] - 0s 72us/step - loss: 3.3002 - acc: 0.5124
Epoch 6/150
242/242 [==============================] - 0s 63us/step - loss: 2.9909 - acc: 0.4917
Epoch 7/150
242/242 [==============================] - 0s 59us/step - loss: 2.5253 - acc: 0.4917
Epoch 8/150
242/242 [==============================] - 0s 71us/step - loss: 2.0864 - acc: 0.5165
Epoch 9/150
242/242 [==============================] - 0s 64us/step - loss: 1.6528 - acc: 0.5744
Epoch 10/150
242/242 [==============================] - 0s 72us/step - loss: 1.2276 - acc: 0.6074
Epoch 11/150
242/242 [========

242/242 [==============================] - 0s 68us/step - loss: 0.5813 - acc: 0.6983
Epoch 85/150
242/242 [==============================] - 0s 55us/step - loss: 0.5581 - acc: 0.7314
Epoch 86/150
242/242 [==============================] - 0s 56us/step - loss: 0.5600 - acc: 0.7231
Epoch 87/150
242/242 [==============================] - 0s 69us/step - loss: 0.5496 - acc: 0.7190
Epoch 88/150
242/242 [==============================] - 0s 61us/step - loss: 0.5584 - acc: 0.7273
Epoch 89/150
242/242 [==============================] - 0s 54us/step - loss: 0.5522 - acc: 0.7355
Epoch 90/150
242/242 [==============================] - 0s 65us/step - loss: 0.5469 - acc: 0.7397
Epoch 91/150
242/242 [==============================] - 0s 73us/step - loss: 0.5503 - acc: 0.7066
Epoch 92/150
242/242 [==============================] - 0s 67us/step - loss: 0.5549 - acc: 0.7355
Epoch 93/150
242/242 [==============================] - 0s 56us/step - loss: 0.5466 - acc: 0.7149
Epoch 94/150
242/242 [===========

242/242 [==============================] - 0s 58us/step - loss: 0.6753 - acc: 0.6942
Epoch 17/150
242/242 [==============================] - 0s 57us/step - loss: 0.6606 - acc: 0.7149
Epoch 18/150
242/242 [==============================] - 0s 58us/step - loss: 0.6591 - acc: 0.7273
Epoch 19/150
242/242 [==============================] - 0s 58us/step - loss: 0.6528 - acc: 0.7107
Epoch 20/150
242/242 [==============================] - 0s 68us/step - loss: 0.6487 - acc: 0.7025
Epoch 21/150
242/242 [==============================] - 0s 58us/step - loss: 0.6496 - acc: 0.7107
Epoch 22/150
242/242 [==============================] - 0s 57us/step - loss: 0.6393 - acc: 0.7231
Epoch 23/150
242/242 [==============================] - 0s 54us/step - loss: 0.6363 - acc: 0.7231
Epoch 24/150
242/242 [==============================] - 0s 58us/step - loss: 0.6267 - acc: 0.7273
Epoch 25/150
242/242 [==============================] - 0s 60us/step - loss: 0.6415 - acc: 0.7025
Epoch 26/150
242/242 [===========

242/242 [==============================] - 0s 59us/step - loss: 0.4110 - acc: 0.8099
Epoch 100/150
242/242 [==============================] - 0s 52us/step - loss: 0.4169 - acc: 0.8182
Epoch 101/150
242/242 [==============================] - 0s 59us/step - loss: 0.4080 - acc: 0.8347
Epoch 102/150
242/242 [==============================] - 0s 58us/step - loss: 0.4089 - acc: 0.8223
Epoch 103/150
242/242 [==============================] - 0s 64us/step - loss: 0.4112 - acc: 0.8264
Epoch 104/150
242/242 [==============================] - 0s 57us/step - loss: 0.4336 - acc: 0.7975
Epoch 105/150
242/242 [==============================] - 0s 61us/step - loss: 0.4037 - acc: 0.8223
Epoch 106/150
242/242 [==============================] - 0s 59us/step - loss: 0.4052 - acc: 0.8347
Epoch 107/150
242/242 [==============================] - 0s 57us/step - loss: 0.4039 - acc: 0.8264
Epoch 108/150
242/242 [==============================] - 0s 57us/step - loss: 0.4056 - acc: 0.8306
Epoch 109/150
242/242 [=

242/242 [==============================] - 0s 56us/step - loss: 0.5872 - acc: 0.7190
Epoch 32/150
242/242 [==============================] - 0s 58us/step - loss: 0.5817 - acc: 0.7107
Epoch 33/150
242/242 [==============================] - 0s 63us/step - loss: 0.5822 - acc: 0.7107
Epoch 34/150
242/242 [==============================] - 0s 68us/step - loss: 0.5765 - acc: 0.7107
Epoch 35/150
242/242 [==============================] - 0s 61us/step - loss: 0.5964 - acc: 0.7107
Epoch 36/150
242/242 [==============================] - 0s 60us/step - loss: 0.5974 - acc: 0.7066
Epoch 37/150
242/242 [==============================] - 0s 62us/step - loss: 0.5719 - acc: 0.7149
Epoch 38/150
242/242 [==============================] - 0s 64us/step - loss: 0.5790 - acc: 0.7066
Epoch 39/150
242/242 [==============================] - 0s 64us/step - loss: 0.5835 - acc: 0.7190
Epoch 40/150
242/242 [==============================] - 0s 61us/step - loss: 0.5640 - acc: 0.7107
Epoch 41/150
242/242 [===========

242/242 [==============================] - 0s 55us/step - loss: 0.4370 - acc: 0.8058
Epoch 115/150
242/242 [==============================] - 0s 52us/step - loss: 0.4346 - acc: 0.8017
Epoch 116/150
242/242 [==============================] - 0s 50us/step - loss: 0.4575 - acc: 0.7893
Epoch 117/150
242/242 [==============================] - 0s 63us/step - loss: 0.4418 - acc: 0.8058
Epoch 118/150
242/242 [==============================] - 0s 58us/step - loss: 0.4407 - acc: 0.8017
Epoch 119/150
242/242 [==============================] - 0s 58us/step - loss: 0.4325 - acc: 0.8058
Epoch 120/150
242/242 [==============================] - 0s 52us/step - loss: 0.4335 - acc: 0.8058
Epoch 121/150
242/242 [==============================] - 0s 56us/step - loss: 0.4287 - acc: 0.8140
Epoch 122/150
242/242 [==============================] - 0s 54us/step - loss: 0.4291 - acc: 0.8099
Epoch 123/150
242/242 [==============================] - 0s 59us/step - loss: 0.4279 - acc: 0.8017
Epoch 124/150
242/242 [=

243/243 [==============================] - 0s 61us/step - loss: 0.5437 - acc: 0.7942
Epoch 47/150
243/243 [==============================] - 0s 61us/step - loss: 0.5465 - acc: 0.7654
Epoch 48/150
243/243 [==============================] - 0s 57us/step - loss: 0.5574 - acc: 0.7737
Epoch 49/150
243/243 [==============================] - 0s 74us/step - loss: 0.5363 - acc: 0.7819
Epoch 50/150
243/243 [==============================] - 0s 60us/step - loss: 0.5405 - acc: 0.7819
Epoch 51/150
243/243 [==============================] - 0s 59us/step - loss: 0.5380 - acc: 0.7819
Epoch 52/150
243/243 [==============================] - 0s 58us/step - loss: 0.5271 - acc: 0.7737
Epoch 53/150
243/243 [==============================] - 0s 58us/step - loss: 0.5240 - acc: 0.7860
Epoch 54/150
243/243 [==============================] - 0s 57us/step - loss: 0.5306 - acc: 0.7901
Epoch 55/150
243/243 [==============================] - 0s 57us/step - loss: 0.5293 - acc: 0.7778
Epoch 56/150
243/243 [===========

243/243 [==============================] - 0s 60us/step - loss: 0.3943 - acc: 0.8107
Epoch 130/150
243/243 [==============================] - 0s 56us/step - loss: 0.3843 - acc: 0.8230
Epoch 131/150
243/243 [==============================] - 0s 59us/step - loss: 0.3806 - acc: 0.8313
Epoch 132/150
243/243 [==============================] - 0s 60us/step - loss: 0.3807 - acc: 0.8230
Epoch 133/150
243/243 [==============================] - 0s 60us/step - loss: 0.3922 - acc: 0.8395
Epoch 134/150
243/243 [==============================] - 0s 62us/step - loss: 0.3837 - acc: 0.8189
Epoch 135/150
243/243 [==============================] - 0s 59us/step - loss: 0.3896 - acc: 0.8230
Epoch 136/150
243/243 [==============================] - 0s 59us/step - loss: 0.3798 - acc: 0.8272
Epoch 137/150
243/243 [==============================] - 0s 61us/step - loss: 0.3834 - acc: 0.8107
Epoch 138/150
243/243 [==============================] - 0s 62us/step - loss: 0.3787 - acc: 0.8395
Epoch 139/150
243/243 [=

243/243 [==============================] - 0s 70us/step - loss: 0.5381 - acc: 0.7119
Epoch 63/150
243/243 [==============================] - 0s 80us/step - loss: 0.5456 - acc: 0.7449
Epoch 64/150
243/243 [==============================] - 0s 76us/step - loss: 0.5425 - acc: 0.7202
Epoch 65/150
243/243 [==============================] - 0s 77us/step - loss: 0.5299 - acc: 0.7490
Epoch 66/150
243/243 [==============================] - 0s 81us/step - loss: 0.5178 - acc: 0.7284
Epoch 67/150
243/243 [==============================] - 0s 86us/step - loss: 0.5254 - acc: 0.7119
Epoch 68/150
243/243 [==============================] - 0s 79us/step - loss: 0.5283 - acc: 0.7407
Epoch 69/150
243/243 [==============================] - 0s 82us/step - loss: 0.5130 - acc: 0.7325
Epoch 70/150
243/243 [==============================] - 0s 81us/step - loss: 0.5143 - acc: 0.7407
Epoch 71/150
243/243 [==============================] - 0s 72us/step - loss: 0.5085 - acc: 0.7407
Epoch 72/150
243/243 [===========

243/243 [==============================] - 0s 60us/step - loss: 0.4013 - acc: 0.8066
Epoch 146/150
243/243 [==============================] - 0s 58us/step - loss: 0.4111 - acc: 0.8189
Epoch 147/150
243/243 [==============================] - 0s 65us/step - loss: 0.3992 - acc: 0.8107
Epoch 148/150
243/243 [==============================] - 0s 65us/step - loss: 0.4063 - acc: 0.8148
Epoch 149/150
243/243 [==============================] - 0s 68us/step - loss: 0.3986 - acc: 0.8066
Epoch 150/150
60/60 [==============================] - 0s 7ms/step


In [0]:
print(f'Results: {results.mean():.2f}, {results.std():.2f}')

Results: 0.77, 0.09


### Our mean accuracy is 0.77 with a std-dev of 0.09. That's not bad for a first shot

In [0]:
# fix random seed for reproducibility
seed = 101
np.random.seed(seed)

def create_model():
  model = Sequential()
  model.add(Dense(13, input_dim=X.shape[1], activation='relu'))
  model.add(Dense(8, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

model = KerasClassifier(build_fn=create_model, verbose=0)
param_grid = {'batch_size': range(32, 128, 32),
              'epochs':     range(20, 200, 20)}

print('Creating grid search')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)

print('Fitting')
grid_result = grid.fit(X, y)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Creating grid search
Fitting


/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: 0.8151815132339402 using {'batch_size': 64, 'epochs': 180}
Means: 0.5445544549537571, Stdev: 0.32498273026560126 with: {'batch_size': 32, 'epochs': 20}
Means: 0.6633663321092184, Stdev: 0.3000055946246005 with: {'batch_size': 32, 'epochs': 40}
Means: 0.570957094922711, Stdev: 0.3273572743345168 with: {'batch_size': 32, 'epochs': 60}
Means: 0.6666666638142992, Stdev: 0.3654260837339631 with: {'batch_size': 32, 'epochs': 80}
Means: 0.21122111876805624, Stdev: 0.2956329437494221 with: {'batch_size': 32, 'epochs': 100}
Means: 0.6105610582694756, Stdev: 0.34514733472950465 with: {'batch_size': 32, 'epochs': 120}
Means: 0.45874587409567125, Stdev: 0.39785922778490446 with: {'batch_size': 32, 'epochs': 140}
Means: 0.39273927422246524, Stdev: 0.32194238291140614 with: {'batch_size': 32, 'epochs': 160}
Means: 0.7458745815572959, Stdev: 0.08978753422069291 with: {'batch_size': 32, 'epochs': 180}
Means: 0.5841584187923091, Stdev: 0.27156821591288166 with: {'batch_size': 64, 'epochs': 20}
Me

### It's getting better - 0.81 accuracy with 180 epochs and batch_size of 64

In [0]:
# fix random seed for reproducibility
seed = 101
np.random.seed(seed)

def create_model(learn_rate=0.01, momentum=0):
  model = Sequential()
  model.add(Dense(13, input_dim=X.shape[1], activation='relu'))
  model.add(Dense(8, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  optimizer = SGD(lr=learn_rate, momentum=momentum)
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model

model = KerasClassifier(build_fn=create_model, verbose=0)
param_grid = {'batch_size': [64],
              'epochs':     [180],
              'learn_rate': [0.001, 0.01, 0.1, 0.2, 0.3],
              'momentum':   [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]}

print('Creating grid search')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)

print('Fitting')
grid_result = grid.fit(X, y)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Creating grid search
Fitting


/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: 0.8580858097611481 using {'batch_size': 64, 'epochs': 180, 'learn_rate': 0.2, 'momentum': 0.8}
Means: 0.528052807837823, Stdev: 0.4503243332742807 with: {'batch_size': 64, 'epochs': 180, 'learn_rate': 0.001, 'momentum': 0.0}
Means: 0.1782178276836282, Stdev: 0.23886222553010986 with: {'batch_size': 64, 'epochs': 180, 'learn_rate': 0.001, 'momentum': 0.2}
Means: 0.34323432225205325, Stdev: 0.42842525954602934 with: {'batch_size': 64, 'epochs': 180, 'learn_rate': 0.001, 'momentum': 0.4}
Means: 0.4587458757677487, Stdev: 0.45433663507435285 with: {'batch_size': 64, 'epochs': 180, 'learn_rate': 0.001, 'momentum': 0.6}
Means: 0.23102310290037603, Stdev: 0.2844536638898839 with: {'batch_size': 64, 'epochs': 180, 'learn_rate': 0.001, 'momentum': 0.8}
Means: 0.11881188472898879, Stdev: 0.14572282646488985 with: {'batch_size': 64, 'epochs': 180, 'learn_rate': 0.001, 'momentum': 0.9}
Means: 0.5412541242322513, Stdev: 0.45433663507435285 with: {'batch_size': 64, 'epochs': 180, 'learn_rate':

### Best: 0.8580858097611481 using {'batch_size': 64, 'epochs': 180, 'learn_rate': 0.2, 'momentum': 0.8}

In [0]:
# fix random seed for reproducibility
seed = 101
np.random.seed(seed)

def create_model(learn_rate=0.01, momentum=0, init_mode='uniform'):
  model = Sequential()
  model.add(Dense(13, input_dim=X.shape[1], kernel_initializer=init_mode, activation='relu'))
  model.add(Dense(8, kernel_initializer=init_mode, activation='relu'))
  model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
  optimizer = SGD(lr=learn_rate, momentum=momentum)
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model

model = KerasClassifier(build_fn=create_model, verbose=0)
param_grid = {'batch_size': [32, 64, 128],
              'epochs':     [160, 180, 200],
              'learn_rate': [0.1, 0.2, 0.3],
              'momentum':   [0.7, 0.8, 0.9],
              'init_mode': ['uniform', 'lecun_uniform', 'normal', 'zero', 
                             'glorot_normal', 'glorot_uniform', 'he_normal', 
                             'he_uniform']}

print('Creating grid search')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)

print('Fitting')
grid_result = grid.fit(X, y)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Creating grid search
Fitting


KeyboardInterrupt: 

### Too late - couldn't finish this run - took too long